In [7]:
import pandas as pd
import numpy as np
import time

# 1. 원본 필드데이터 갖고오기 
start_time = time.time() 
df = pd.read_excel('data/field.xlsx')

# 1-1.NaN만 있는 열 제거 / 열 이름 모든 공백 제거
df = df.dropna(axis=1, how='all') 
df.columns = df.columns.str.replace(' ', '').str.replace('\n','',regex=True).str.replace('\r','',regex=True)
#-------------------------------------------------------------------------------------------------------------------------------
# 2.'엔진번호/엔진생산일.csv 파워쿼리 데이터' 가져오기

### 2-1) '엔진생산일' 열 에서 NaN일때 '엔진번호/엔진생산일 파워쿼리 데이터'의 값으로 채우기

# 엔진생산일이 NaN인 위치
mask = df['엔진생산일'].isna()
### 2-2) '엔진생산일'이 NaN일 때 '생산년월일'의 값으로 채우기
df['엔진생산일'] = df['엔진생산일'].fillna(df['생산년월일'])
# '생산년월일' 값에서 '/' 제거 (YYYY/MM/DD → YYYYMMDD)
df.loc[mask, '엔진생산일'] = df.loc[mask, '생산년월일'].str.replace('/', '')

### >>> '엔진생산일' 정수화 작업 추가!
#df['엔진생산일'] = pd.to_numeric(df['엔진생산일'], errors='coerce').astype('Int64')
#-------------------------------------------------------------------------------------------------------------------------------

# 3.'엔진생산월' 열 추가 (YYYY/MM 형태)
df['엔진생산월'] = df['엔진생산일'].astype(str).str.slice(0,6).apply(
    lambda x: f"{x[:4]}/{x[4:6]}" if len(x) == 6 and x.isdigit() else '')

end_time = time.time()            # 종료 시간 저장
print(f"소요 시간: {end_time - start_time:.2f}초")

#-------------------------------------------------------------------------------------------------------------------------------
# 4. 품번 기준 '품명수정' 열 추가
  # 품번을 기준으로 품명수정 열 붙이기
  start_time = time.time() 
component_df = pd.read_excel('data/component.xlsx')  # 품번, 품명수정 포함
  # component.xlsx의 열 이름 공백도 정리하세요!
component_df.columns = component_df.columns.str.replace(' ', '')

  # 품번 기준 merge (left join)
df.set_index('부품번호', inplace=True)
component_df.set_index('부품번호', inplace=True)
df = df.join(component_df[['품명수정']])
df.reset_index(inplace=True)
end_time = time.time()            # 종료 시간 저장
print(f"소요 시간: {end_time - start_time:.2f}초")
#-------------------------------------------------------------------------------------------------------------------------------
# 5. '엔진기종' 열 추가

# engine_type 데이터 불러오기
engine_type_df = pd.read_excel('data/engine_type.xlsx')  # 엔진4자리, 엔진기종 포함
# 열 이름 공백도 정리
engine_type_df.columns = engine_type_df.columns.str.replace(' ', '')
# 원본에서 엔진번호 앞 4자리 추출
df['엔진4자리'] = df['엔진넘버'].astype(str).str.slice(0, 4)
# 앞4자리 기준 merge (left join)
df = df.merge(engine_type_df[['엔진4자리','엔진기종']], on='엔진4자리', how='left')

###print(df.columns)
###print(df[['품명수정','엔진기종']].head())
#-------------------------------------------------------------------------------------------------------------------------------
#6.중복값 제거
###6-1. 'RO번호', '차대번호', '총주행거리(Km)'를 합쳐서 중복확인 열 만들기
df['중복확인'] = df['RO번호'].astype(str) + '_' + df['차대번호'].astype(str) + '_' + df['총주행거리(Km)'].astype(str)
##6-2. '중복확인' 컬럼 기준, 중복값이 있는(즉 2개 이상인) 행 전부 삭제
df = df[~df.duplicated('중복확인', keep='first')]
##6-3. 필요하면 중복확인 열 제거 (선택)
# df = df.drop('중복확인', axis=1)
# 중복값 남았는지 확인. 중복없으면 False
print((df['중복확인'].value_counts() > 1).any())
#-------------------------------------------------------------------------------------------------------------------------------

df.to_excel('data/field_final.xlsx', index=False)


IndentationError: unexpected indent (3903914701.py, line 38)